In [5]:
from IPython import display
import PIL.Image
from IPython.display import clear_output
from imutils.video import FPS

In [6]:
import sys

import socket
import traceback
import cv2
from imutils.video import VideoStream
import imagezmq
import threading
import numpy as np
from time import sleep

class VideoStreamSubscriber:

    def __init__(self, hostname, port):
        self.hostname = hostname
        self.port = port
        self._stop = False
        self._data_ready = threading.Event()
        self._thread = threading.Thread(target=self._run, args=())
        self._thread.daemon = True
        self._thread.start()

    def receive(self, timeout=15.0):
        flag = self._data_ready.wait(timeout=timeout)
        if not flag:
            raise TimeoutError(
                "Timeout while reading from subscriber tcp://{}:{}".format(self.hostname, self.port))
        self._data_ready.clear()
        return self._data

    def _run(self):
        receiver = imagezmq.ImageHub("tcp://{}:{}".format(self.hostname, self.port), REQ_REP=False)
        while not self._stop:
            self._data = receiver.recv_jpg()
            self._data_ready.set()
        receiver.close()

    def close(self):
        self._stop = True

# Simulating heavy processing load
def limit_to_2_fps():
    sleep(0.5)

In [12]:

hostname = "192.168.0.195" 
port = 5555
receiver = VideoStreamSubscriber(hostname, port)
fps = FPS().start()
try:
    for i in range(10):
        msg, frame = receiver.receive()
        image = cv2.imdecode(np.frombuffer(frame, dtype='uint8'), -1)
        fps.update()
        im=PIL.Image.fromarray(image)
        display.display(im)
        clear_output(wait=True)
except Exception as e:
    pass
    
fps.stop()
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

[INFO] approx. FPS: 0.31


In [13]:
1/0.31


3.2258064516129035